# BAX pore assembly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

%matplotlib inline

## Tree tensor integrator

In [ ]:
tree = readTree("output/bax_new/output_t2900.nc")
tree.calculateObservables(np.array([12, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0]))

dx0 = np.prod(tree.root.child[0].grid.n[:2])
X0_sum = np.zeros((dx0, tree.root.child[0].X.shape[1]))
for i in range(dx0):
    for j in range(tree.root.child[0].X.shape[1]):
        X0_sum[i, j] = np.sum(tree.root.child[0].X[i::dx0,j])

P_sum_tensor = X0_sum @ tree.root.Q[:, :, 0] @ tree.root.child[1].X_sum

X0_slice = tree.root.child[0].X[9 * tree.root.child[0].grid.n[0] + 2 * np.prod(tree.root.child[0].grid.n[:2]) : 10 * tree.root.child[0].grid.n[0] + 2 * np.prod(tree.root.child[0].grid.n[:2]), :]

P_slice_tensor = X0_slice @ tree.root.Q[:, :, 0] @ tree.root.child[1].X_slice

## Matrix integrator

In [ ]:
with xr.open_dataset("output/bax_general_230730_r5_so_tau1e0_ss100/output_t100.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    slice_vec = np.array([17, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0])
    idx_2D = np.array([0, 1])
    P_slice_matrix = lr_sol.slicedDistributions(slice_vec)

## SSA

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_general_ref_1e7.npy")
ssa_sol = SSASol(result)
_, _, P_slice_ssa, _ = ssa_sol.calculateObservables(slice_vec, idx_2D)

## Plots

In [ ]:
plt.figure(2)
plt.plot(np.arange(tree.root.child[0].grid.n[0])[10:26], P_slice_tensor[10:26], 'rx', label="Hierarchical DLR approx.")
plt.plot(np.arange(grid.n[0])[10:26], P_slice_matrix[0][10:26], 'bo', label="DLR approx.")
plt.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0]), P_slice_ssa[-1][0], 'gs', label="SSA ($10^7$ runs)")
plt.legend()
plt.yscale("log")
plt.xlabel("$x_0$")
plt.ylabel("$P_\\mathrm{{S}}(x_0)$")
plt.xlim([9, 26])
plt.savefig("plots/hierarchical_bax_plot1.pdf")